In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector
import sys

from app.objects.DatabaseUtils import par_db

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

from sklearn.ensemble import RandomForestClassifier

In [28]:
par_db

{'host': 'db', 'database': 'streamapp', 'user': 'root', 'password': 'root'}

In [24]:
mydb = mysql.connector.connect(
            host=par_db.get("host"),
            user=par_db.get("user"),
            database=par_db.get("database"),
            password=par_db.get("password"),
        )
cursor = mydb.cursor() 
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall)
colnames = [desc[0] for desc in cur.description]
df.columns = colnames


DatabaseError: 2005 (HY000): Unknown MySQL server host 'db' (-3)

In [3]:
df.head(5)

,id,x,y,z,date_mpu,time_mpu,time_float
0,101,-1412,1064,15756,2022-03-25,20:20:59,20.3497
1,102,-1232,1000,15732,2022-03-25,20:21:16,20.3544
2,103,-1112,1000,15648,2022-03-25,20:21:32,20.3589
3,104,-1164,1000,15684,2022-03-25,20:21:48,20.3633
4,105,-1196,892,15768,2022-03-25,20:22:03,20.3675


In [4]:
df2.head(5)

,id,x,y,z,date_mpu,time_mpu,time_float
0,1,-852,-6288,13332,2022-03-24,17:08:00,17.1333
1,2,-11240,-4752,11148,2022-03-24,17:08:18,17.1383
2,3,288,-1656,8444,2022-03-24,17:08:33,17.1425
3,4,6144,-4272,17420,2022-03-24,17:08:49,17.1469
4,5,1420,900,15400,2022-03-24,17:09:05,17.1514


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          60 non-null     int64  
 1   x           60 non-null     int64  
 2   y           60 non-null     int64  
 3   z           60 non-null     int64  
 4   date_mpu    60 non-null     object 
 5   time_mpu    60 non-null     object 
 6   time_float  60 non-null     float64
dtypes: float64(1), int64(4), object(2)
memory usage: 3.4+ KB


In [6]:
df["is_abrupt"] = "0"
df2["is_abrupt"] = "1"

In [7]:
df2.head(5)

,id,x,y,z,date_mpu,time_mpu,time_float,is_abrupt
0,1,-852,-6288,13332,2022-03-24,17:08:00,17.1333,1
1,2,-11240,-4752,11148,2022-03-24,17:08:18,17.1383,1
2,3,288,-1656,8444,2022-03-24,17:08:33,17.1425,1
3,4,6144,-4272,17420,2022-03-24,17:08:49,17.1469,1
4,5,1420,900,15400,2022-03-24,17:09:05,17.1514,1


In [8]:
df = pd.concat([df,df2]).sort_index()
df = df.reset_index(drop=True)
df.head(5)

,id,x,y,z,date_mpu,time_mpu,time_float,is_abrupt
0,101,-1412,1064,15756,2022-03-25,20:20:59,20.3497,0
0,1,-852,-6288,13332,2022-03-24,17:08:00,17.1333,1
1,2,-11240,-4752,11148,2022-03-24,17:08:18,17.1383,1
1,102,-1232,1000,15732,2022-03-25,20:21:16,20.3544,0
2,103,-1112,1000,15648,2022-03-25,20:21:32,20.3589,0


In [12]:
#Separando o X e o y do modelo
X = df.drop(columns=["is_abrupt", "time_mpu", "date_mpu"])

y = df['is_abrupt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

#Escalando os dados

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [15]:
#Random Forest
def rfc(x_test, x_train, y_test, y_train):
  clf=RandomForestClassifier(n_estimators=100)
  clf.fit(x_train, y_train)
  print(f"Accuracy score: {round( accuracy_score(clf.predict(x_test), y_test), 2)}")

In [14]:
runRegr(x_train_scaled, x_test_scaled, y_train, y_test)

MSE: 0.0
